<a href="https://colab.research.google.com/github/Sihan-A/transformers_NLP/blob/main/03_RoBERTa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3 RoBERTa

1. RoBERTa- and DistilBERT-like models
1. How to train a tokenizer from scratch
1. Byte-level byte-pair encoding
1. Saving the trained tokenizer to files
1. Recreating the tokenizer for the pretraining process
1. Initializing a RoBERTa model from scratch
1. Exploring the configuration of the model
1. Exploring the 80 million parameters of the model
1. Building the dataset for the trainer
1. Initializing the trainer
1. Pretraining the model
1. Saving the model
1. Applying the model to the downstream tasks of masked language modeling

## Step 1: Loading the dataset

In [ ]:
!curl -L https://raw.githubusercontent.com/PacktPublishing/Transformers-for-Natural-Language-Processing/master/Chapter03/kant.txt --output "kant.txt"

## Step 2: Installing Hugging Face transformers

In [2]:
!pip uninstall -y tensorflow
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'

Found existing installation: tensorflow 2.6.0
Uninstalling tensorflow-2.6.0:
  Successfully uninstalled tensorflow-2.6.0
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-j33qp6g9
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-j33qp6g9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.3 MB 6.1 MB/s 
     |████████████████████████████████| 636 kB 66.6 MB/s 
     |████████████████████████████████| 895 kB 58.1 MB/s 
     |████████████████████████████████| 52 kB 2.0 MB/s 
  Created wheel for transformers: filename=transformers-4.11.0.dev0-py3-none-any.whl size=2832884 sha256=c1b5ff808b1b6a9f60ccabf95d48e87dea852d77468ccb0681c010ba53de32c8
  Stored in directory: /tmp/pip-ephem-wheel-cache-9mm8rz3c/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers
  Attemp

## Step 3: Training a tokenizer

In [3]:
%%time
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer
paths = [str(x) for x in Path(".").glob("**/*.txt")]

tokenizer = ByteLevelBPETokenizer()
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2,
                special_tokens=["<s>", "<pad>", "</s>", "<unk>", "<mask>"])

CPU times: user 5.22 s, sys: 187 ms, total: 5.41 s
Wall time: 2.81 s


## Step 4: Saving the files to disk

In [5]:
import os
token_dir = "/content/KantaiBERT"
if not os.path.exists(token_dir):
    os.makedirs(token_dir)
tokenizer.save_model("KantaiBERT")

['KantaiBERT/vocab.json', 'KantaiBERT/merges.txt']

## Step 5: Loading the trained tokenizer files

In [6]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

tokenizer = ByteLevelBPETokenizer("./KantaiBERT/vocab.json",
                                  "./KantaiBERT/merges.txt")

In [8]:
tokenizer.encode("The Critique of Pure Reason.").tokens

['The', 'ĠCritique', 'Ġof', 'ĠPure', 'ĠReason', '.']

In [9]:
tokenizer.encode("The Critique of Pure Reason.")

Encoding(num_tokens=6, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [11]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [12]:
tokenizer.encode("The Critique of Pure Reason.")

Encoding(num_tokens=8, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [13]:
tokenizer.encode("The Critique of Pure Reason.").tokens

['<s>', 'The', 'ĠCritique', 'Ġof', 'ĠPure', 'ĠReason', '.', '</s>']

## Step 6: Checking resource constraints: GPU and CUDA

In [14]:
!nvidia-smi

Fri Sep 17 05:04:15 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [15]:
import torch
torch.cuda.is_available()

True

## Step 7: Defining the configuration of the model

In [16]:
from transformers import RobertaConfig
config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [19]:
print(config)

RobertaConfig {
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.11.0.dev0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 52000
}



## Step 8: Reloading the tokenizer in transformers

In [21]:
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained("./KantaiBERT", max_length=512)

file ./KantaiBERT/config.json not found
